In [7]:
import numpy as np
np.printoptions(suppress= True)

In [8]:
def normalKaczmarz(A, b, l = 1.0, MAX_TOL= 1e-16, MAX_ITER= 1000):
    """
    Implement the Kaczmarz algorithm to solve a system of linear equation.

    The algorithm works under the principle of finding an estimated solution (xhat) that satisfies 
    ||A @ xhat - b||^2 < e, with e is the tolerance value.
    """
    assert (0 <= l <= 1), "The relaxation parameter must be of the range 0 < l < 1!"
    
    X = np.random.normal(size= (A.shape[1], 1))
    b = np.reshape(b, (A.shape[0], 1))

    iter = 0
    residual = 1e+6
    while (residual > MAX_TOL) and (iter < MAX_ITER):
        idx = iter % A.shape[0]
        a_i = A[idx, :]
        b_i = b[idx]

        change = np.reshape(l * np.multiply(b_i - np.dot(a_i, X), a_i) / np.power(np.linalg.norm(a_i), 2),
                            (A.shape[1],1))
        Xnew = X + change
        residual = np.linalg.norm(b - np.dot(A, Xnew))
        X = np.copy(Xnew)
        iter += 1

    return X

def randomizedKaczmarz(A, b, l = 1.0, MAX_TOL= 1e-16, MAX_ITER= 1000):
    """
    Implement the Kaczmarz algorithm to solve a system of linear equation.

    The algorithm works under the principle of finding an estimated solution (xhat) that satisfies 
    ||A @ xhat - b||^2 < e, with e is the tolerance value.
    """
    assert (0 <= l <= 1), "The relaxation parameter must be of the range 0 < l < 1!"

    X = np.random.normal(size= (A.shape[1], 1))
    b = np.reshape(b, (A.shape[0], 1))
    probs = np.power(np.linalg.norm(A, axis = 1) / np.linalg.norm(A), 2)

    iter = 0
    residual = 1e+6
    while (residual > MAX_TOL) and (iter < MAX_ITER):
        idx = np.random.choice(A.shape[0], p = probs)
        a_i = A[idx, :]
        b_i = b[idx]

        change = np.reshape(l * np.multiply(b_i - np.dot(a_i, X), a_i) / np.power(np.linalg.norm(a_i), 2),
                            (A.shape[1],1))
        Xnew = X + change
        residual = np.linalg.norm(b - np.dot(A, Xnew))
        X = np.copy(Xnew)
        iter += 1

    return X

def admm(d, A, b, rho= 1, max_iter= 1000):
    """
    Implement the alternating direction method of multipliers (ADMM) optimization algorithm to solve a linear program.
    The implementation is based on the formulas mentioned in Liu et al. (2018).

    The linear program has the following form: min d^Tx, s.t. Ax = b
    """
    d = d.T
    b = b.T

    # Extract the number of variables and constraints from the matrix dimensions
    no_of_vars = A.shape[1]
    no_of_constraints = A.shape[0]

    # Initialize the dual variables y and mu
    y  = np.random.normal(size= (1,no_of_vars))
    mu = np.random.normal(size= (1,no_of_vars))

    # Construct the augmented Lagrangian matrix
    C = np.block([
        [rho * np.eye(no_of_vars), A.T],
        [A, np.zeros((no_of_constraints,no_of_constraints))]
                ])

    for i in range(max_iter):
        # Solve for the primal variable x
        alpha = y - (1/rho) * (mu + d)
        m = np.block([rho * alpha, b]).T
        x = normalKaczmarz(C, m)[: no_of_vars]

        # Apply soft-thresholding to update the dual variable y
        beta = x + (1/rho) * mu.T
        beta[beta < 0] = 0
        y = beta.T

        # Update the dual variable mu
        mu = mu + rho * (x.T - y)

    return (x,y,mu)

In [9]:
d = np.array([-1, -2, -4, 0, 0, 0])
A = np.array([
    [3, 1, 5, 1, 0, 0],
    [1, 4, 1, 0, 1, 0],
    [2, 0, 2, 0, 0, 1]])
b = np.array([10, 8, 7])

x, _ , _ = admm(d, A, b)
print(x)

[[ 2.05017231e-05]
 [ 1.57894048e+00]
 [ 1.68424878e+00]
 [-3.76536906e-05]
 [ 1.32870161e-05]
 [ 3.63144096e+00]]
